In [1]:
"""
This module contains functions for loading a pdb file and calculating
the atomic density fields for different atom types. The fields can then be
used for plotting, or to send into the convolutional neural network.
"""

import os
import sys
import torch
import numpy as np

from biopandas.pdb import PandasPdb
from biopandas.mol2 import PandasMol2

import pickle

import numpy as np

import pybel
from math import ceil, sin, cos, sqrt, pi
from itertools import combinations

In [75]:
max_charge = []
min_charge = []
file_name = []

# for filename in os.listdir('./pocket_mol2'):
#     mol = PandasMol2().read_mol2('./pocket_mol2/' + filename)
#     partial_charge = mol.df['charge'].values
    
    
#     max_charge.append(np.max(partial_charge))
#     min_charge.append(np.min(partial_charge))

for filename in os.listdir('./ligand_mol2'):

    mol = PandasMol2().read_mol2('./ligand_mol2/' + filename)
    
    partial_charge = mol.df['charge'].values
    
    file_name.append(filename)
    max_charge.append(np.max(partial_charge))
    min_charge.append(np.min(partial_charge))


max_test = np.max(max_charge)
min_test = np.min(min_charge)
print(max_test, min_test)
    
    

10.0 -5.0


In [19]:
minpos = min_charge.index(min(min_charge))
maxpos = max_charge.index(max(max_charge))



max_file = file_name[maxpos]
min_file = file_name[minpos]

print(min_file, max_file)

5zeq_ligand.mol2 3p44_ligand.mol2


In [76]:
j = 0
file_exclude = []

for i, (a, b) in enumerate(zip(min_charge, max_charge)):
    
    
    if a < -1.167 or b > 2.276:
        file_exclude.append(file_name[i])
        j = j+1
#         print(file_name[i])
#     if b > 2.276:
#         j = j+1
        #print(file_name[i])
        
print(j)

184


In [77]:
file_exclude

['6dud_ligand.mol2',
 '5tg7_ligand.mol2',
 '5u48_ligand.mol2',
 '5waf_ligand.mol2',
 '3vjt_ligand.mol2',
 '4u0x_ligand.mol2',
 '3sl1_ligand.mol2',
 '4qvl_ligand.mol2',
 '2y4a_ligand.mol2',
 '2jld_ligand.mol2',
 '5tgy_ligand.mol2',
 '4hww_ligand.mol2',
 '3fkv_ligand.mol2',
 '4wkv_ligand.mol2',
 '5fqb_ligand.mol2',
 '4q3s_ligand.mol2',
 '3zju_ligand.mol2',
 '4iu0_ligand.mol2',
 '4daw_ligand.mol2',
 '3o86_ligand.mol2',
 '5w14_ligand.mol2',
 '3nzi_ligand.mol2',
 '4qw1_ligand.mol2',
 '3mnu_ligand.mol2',
 '3way_ligand.mol2',
 '3mmr_ligand.mol2',
 '3ixg_ligand.mol2',
 '2cfd_ligand.mol2',
 '5agj_ligand.mol2',
 '4wyy_ligand.mol2',
 '3cst_ligand.mol2',
 '4i60_ligand.mol2',
 '5ne1_ligand.mol2',
 '2y59_ligand.mol2',
 '1lpg_ligand.mol2',
 '6d1m_ligand.mol2',
 '1sqa_ligand.mol2',
 '5zeq_ligand.mol2',
 '1k1i_ligand.mol2',
 '4kai_ligand.mol2',
 '3zp9_ligand.mol2',
 '4qvv_ligand.mol2',
 '5wac_ligand.mol2',
 '6rsa_ligand.mol2',
 '2y2h_ligand.mol2',
 '4len_ligand.mol2',
 '5nxx_ligand.mol2',
 '4wz5_ligand

In [78]:
def load_pdb(path):
    """
    Loads all of the atomic positioning/type arrays from a pdb file.
    The arrays can then be transformed into density (or "field") tensors before
        being sent through the neural network.
    Parameters:
        path (str, required): The full path to the pdb file being voxelized.
    Returns:
        dictionary: A dictionary containing the following arrays from
            the pdb file: num_atoms, atom_types, positions, atom_type_set,
            xcoords, ycoords, zcoords, residues, residue_set
    """
    
    file_type = path.split('.')[-1]
    
    if file_type == 'pdb':
    #pdb = PandasPdb().read_pdb(path)
        pdb = PandasPdb().read_pdb(path)
    # This just creates a dataframe from the pdb file using biopandas
    #print('This is vars',vars(pdb))
        pdf = pdb.df['ATOM']
        x_coords = pdf['x_coord'].values
        y_coords = pdf['y_coord'].values
        z_coords = pdf['z_coord'].values
        atom_types = pdf['atom_name'].values
        residue_names = pdf['residue_name'].values
    
    elif file_type == 'mol2':
        mol = PandasMol2().read_mol2(path)
        pdf = mol
        x_coords = pdf.df['x'].values
        y_coords = pdf.df['y'].values
        z_coords = pdf.df['z'].values
        atom_types = pdf.df['atom_name'].values
        residue_names = pdf.df['subst_name'].values
        partial_charge = pdf.df['charge'].values
        smarts_notation = next(pybel.readfile(file_type, path))
    else:
        raise ValueError('Need a pdb or mol2 file')
    # atomic coordinates
   

    # create an array containing tuples of x,y,z for every atom
    positions = []
    for i, x in enumerate(x_coords):
        position_tuple = (x_coords[i], y_coords[i], z_coords[i])
        positions.append(position_tuple)
    positions = np.array(positions)

    # names of all the atoms contained in the protein
    
    num_atoms = len(atom_types)
    atom_type_set = np.unique(atom_types)
    num_atom_types = len(atom_type_set)

    # residue names
    
    residue_set = np.unique(residue_names)

    protein_dict = {'x_coords':x_coords, 'y_coords':y_coords, 'z_coords':z_coords,
                    'positions':positions, 'atom_types':atom_types,
                    'num_atoms':num_atoms, 'atom_type_set':atom_type_set,
                    'num_atom_types':num_atom_types, 'residues':residue_names,
                    'residue_set':residue_set}
    
    if file_type == 'mol2':
        protein_dict['charge'] = partial_charge
        protein_dict['smarts'] = smarts_notation

    # add a value to the dictionary, which is all of the atomic coordinates just
    # shifted to the origin
    protein_dict = shift_coords(protein_dict)

    return protein_dict

def shift_coords(protein_dict):
    """
    This function shifts the coordinates of a protein so that it's coordinates are in the center
    of the field tensor.
    Parameters:
        protein_dict (dict): A dictionary of information from the first part of the load_pdb function.
    Returns:
        dictionary: The original protein dict but with an added value containing
            the coordinates of the protein shifted to the origin.
    """
    # find the extreme x, y, and z values that exist in the protein atomic coordinates
    x_extremes = np.array([protein_dict['x_coords'].min(), protein_dict['x_coords'].max()])
    y_extremes = np.array([protein_dict['y_coords'].min(), protein_dict['y_coords'].max()])
    z_extremes = np.array([protein_dict['z_coords'].min(), protein_dict['z_coords'].max()])

    # calculate the midpoints of the extremes
    midpoints = [np.sum(x_extremes)/2, np.sum(y_extremes)/2, np.sum(z_extremes)/2]
#     print(x_extremes, y_extremes, z_extremes)

    # shift the coordinates by the midpoints of those extremes (center the protein on the origin)
    protein_dict['shifted_positions'] = protein_dict['positions'] - midpoints

    return protein_dict

In [79]:
pro_dict = load_pdb('1a1e_ligand.mol2')
print(pro_dict['smarts'])

C(=O)(C)N[C@H](C(=O)N[C@H](C(=O)N1CCC[C@H](C1)CCCC)CCC(=O)O)Cc1ccc(cc1)OP(O)(O)O	1a1e_ligand



In [85]:
def grid_positions(grid_array):
    """
    This function returns the 3D meshgrids of x, y and z positions.
    These cubes will be flattened, then used as a reference coordinate system
    to place the actual channel densities into.
    Parameters:
        grid_positions (pytorch tensor): lineraly spaced grid
    Returns:
        array: meshgrid array of the x, y and z positions.
    """
    xgrid = grid_array.view(-1, 1, 1).repeat(1, len(grid_array), len(grid_array))
    ygrid = grid_array.view(1, -1, 1).repeat(len(grid_array), 1, len(grid_array))
    zgrid = grid_array.view(1, 1, -1).repeat(len(grid_array), len(grid_array), 1)
    return (xgrid, ygrid, zgrid)

def make_fields(protein_dict, channels, bin_size, num_bins):
    """
    This function takes a protein dict (from load_pdb function) and outputs a
        large tensor containing many atomic "fields" for the protein.
    The fields describe the atomic "density" (an exponentially decaying function
        of number of atoms in a voxel) of any particular atom type.
    Parameters:
        protein_dict (dict, requred): dictionary from the load_pdb function
        channels (list-like, optional): the different atomic densities we want fields for
            theoretically these different fields provide different chemical information
            full list of available channels is in protein_dict['atom_type_set']
        bin_size (float, optional): the side-length (angstrom) of a given voxel in the box
            that atomic densities are placed in
        num_bins (int, optional): how big is the cubic field tensor side length
            (i.e., num_bins is box side length)
    Returns:
        dictionary: A list of atomic density tensors (50x50x50), one for each
            channel in channels
    """
    # sets of allowed filters to build channels with
    residue_filters = protein_dict['residue_set']
    atom_filters = protein_dict['atom_type_set']
    
    general_filters = ['all_C', 'all_O', 'all_N']
    residue_property_filters = np.array(['acidic', 'basic', 'polar', 'nonpolar',\
                                         'charged', 'amphipathic'])
    smart_filters = np.array(['hydrophobic', 'aromatic', 'acceptor', 'donor',
                             'ring'])
    named_prop = np.array(['hyb', 'heavyvalence', 'heterovalence', 'partialcharge'])
    other_filters = np.array(['backbone', 'sidechains'])

    # consolidate into one set of filters
    filter_set = {'atom':atom_filters, 'residue':residue_filters,\
                  'residue_property':residue_property_filters, \
                  'smarts_property':smart_filters, \
                  'atom_property': named_prop, 'general': general_filters,
                  'other':other_filters}

    # construct a single empty field, then initialize a dictionary with one
    # empty field for every channel we are going to calculate the density for
    empty_field = torch.zeros((num_bins, num_bins, num_bins))
    fields = {channel:empty_field for channel in channels}

    # create linearly spaced grid (default is -49 to 49 in steps of 2)
    grid_1d = torch.linspace(start=-num_bins / 2 * bin_size + bin_size / 2,
                             end=num_bins / 2 * bin_size - bin_size / 2,
                             steps=num_bins)

    # This makes three 3D meshgrids in for the x, y, and z positions
    # These cubes will be flattened, then used as a reference coordinate system
    # to place the actual channel densities into
    xgrid, ygrid, zgrid = grid_positions(grid_1d)

    for channel_index, channel in enumerate(channels):

        # no illegal channels allowed, assume the channel sucks
        channel_allowed = check_channel(channel, filter_set)

        if channel_allowed:
            pass
        else:
            #err_string = 'Allowed channels are: in a protein\'s atom_type_set,
                        # residue_set',or the \'sidechains\' and \'backbone\' channels.'
            raise ValueError('The channel ', channel, ' is not allowed for this protein.')


        # Extract positions of atoms that are part of the current channel

        atom_positions = find_channel_atoms(channel, protein_dict, filter_set)
        if channel in filter_set['atom']:
            print('atom')
        
        #print('This is channel ', atom_positions)

        atom_positions = torch.FloatTensor(atom_positions)
        

        # xgrid.view(-1, 1) is 125,000 long, because it's viewing a 50x50x50 cube in one column
        # then you repeat that column horizontally for each atom
        xx_xx = xgrid.view(-1, 1).repeat(1, len(atom_positions))
        yy_yy = ygrid.view(-1, 1).repeat(1, len(atom_positions))
        zz_zz = zgrid.view(-1, 1).repeat(1, len(atom_positions))
        
        # at this point we've created 3 arrays that are 125,000 long
        # and as wide as the number of atoms that are the current channel type
        # these 3 arrays just contain the flattened x,y,z positions of our 50x50x50 box


        # now do the same thing as above, just with the ACTUAL atomic position data
        posx_posx = atom_positions[:, 0].contiguous().view(1, -1).repeat(len(xgrid.view(-1)), 1)
        print(xx_xx[0].shape)
        posy_posy = atom_positions[:, 1].contiguous().view(1, -1).repeat(len(ygrid.view(-1)), 1)
        posz_posz = atom_positions[:, 2].contiguous().view(1, -1).repeat(len(zgrid.view(-1)), 1)
        # three tensors of the same size, with actual atomic coordinates

        # normalizes the atomic positions with respect to the center of the box
        # and calculates density of atoms in each voxel
        sigma = 0.5*bin_size

        
        if channel in named_prop:
            prop = []
            for atom in protein_dict['smarts']:
    #atom.__getattribute__(prop)
                prop.append(atom.__getattribute__(named_prop[np.int(np.where(named_prop == channel)[0])]))

            prop = np.array(prop)
            print(max(prop))

            normalized_prop =  (prop-min(prop))/(max(prop)-min(prop))
            #print(normalized_prop)
            
            density = torch.exp(-(((xx_xx - posx_posx)**2) * normalized_prop
                              + ((yy_yy - posy_posy)**2) * normalized_prop
                              + ((zz_zz - posz_posz)**2) * normalized_prop) / (2 * (sigma)**2))
        
        else:
            density = torch.exp(-((xx_xx - posx_posx)**2
                            + (yy_yy - posy_posy)**2
                            + (zz_zz - posz_posz)**2) / (2 * (sigma)**2))
        print(density.shape)

        # Normalize so each atom density sums to one
        density /= torch.sum(density, dim=0)

        # Sum densities and reshape to original shape
        sum_densities = torch.sum(density, dim=1).view(xgrid.shape)
        print(sum_densities.shape)

        # set all nans to 0
        sum_densities[sum_densities != sum_densities] = 0

        # add two empty dimmensions to make it 1x1x50x50x50, needed for CNN
        # sum_densities = sum_densities.unsqueeze(0)
        # sum_densities = sum_densities.unsqueeze(0)

        #fields[atom_type_index] = sum_densities
        fields[channel] = sum_densities.numpy()

#     if return_bins:
#         return fields, num_bins
#     else:
        return fields

def check_channel(channel, filter_set):
    """
    This function checks to see if a channel the user is asking to make a field
        for is an allowed channel to ask for.
    Parameters:
        channel (str, required): The atomic channel being requested
        filter_set (dict, required): The set of defined atomic filters
    Returns:
        boolean: indicator for if the channel is allowed
    """
    channel_allowed = False
    for key in filter_set:
        if channel in filter_set[key]:
            channel_allowed = True

    return channel_allowed

def find_channel_atoms(channel, protein_dict, filter_set):
    """
    This function finds the coordinates of all relevant atoms in a channel.
    It uses the filter set to constrcut the atomic channel (i.e., a channel can
        be composed of multiple filters).
    Parameters:
        channel (str, required): The atomic channel being constructed
        protein_dict (dict, required): The dictionary of the protein, returned from
            load_pdb()
        filter_set (dict, required): The set of available filters to construct channels with
    Returns:
        numpy array:  array containing the coordinates of each atom that is relevant
            to the channel
    """
    if channel in filter_set['atom']:
        atom_positions = protein_dict['shifted_positions'][protein_dict['atom_types'] == channel]
    
    elif channel in filter_set['general']:
        atom_dict = {'all_C' :'C', 'all_O' : 'O', 'all_N' : 'N'}
        atom_positions = protein_dict['shifted_positions']\
                        [[a.startswith(atom_dict[channel], 0) for a in protein_dict['atom_types']]]
        

    elif channel in filter_set['residue']:
        atom_positions = protein_dict['shifted_positions'][protein_dict['residues'] == channel]
        
    elif channel in filter_set['smarts_property']:
        smarts_list = [
                '[#6+0!$(*~[#7,#8,F]),SH0+0v2,s+0,S^3,Cl+0,Br+0,I+0]',
                '[a]',
                '[!$([#1,#6,F,Cl,Br,I,o,s,nX3,#7v5,#15v5,#16v4,#16v6,*+1,*+2,*+3])]',
                '[!$([#6,H0,-,-2,-3]),$([!H0;#7,#8,#9])]',
                '[r]'
            ]
        smarts_list_entry = smarts_list[np.int(np.where(filter_set['smarts_property'] == channel)[0])]
        pattern = pybel.Smarts(smarts_list_entry)
        atoms_smart = np.array(list(*zip(*pattern.findall(protein_dict['smarts']))),
                                    dtype=int) - 1
        print(atoms_smart)
        atom_positions = protein_dict['shifted_positions'][atoms_smart]
    
    elif channel in filter_set['atom_property']:
        atom_positions = protein_dict['shifted_positions']

    elif channel in filter_set['other']: # backbone or sidechain
        if channel == 'backbone':
            # create boolean arrays for backbone atoms
            bool_oxygen = protein_dict['atom_types'] == 'O'
            bool_carbon = protein_dict['atom_types'] == 'C'
            bool_alphacarbon = protein_dict['atom_types'] == 'CA'
            bool_nitrogen = protein_dict['atom_types'] == 'N'

            # sum of all the backbone channels into one boolean array
            bool_backbone = bool_oxygen + bool_carbon + bool_alphacarbon + bool_nitrogen

            # select the backbone atoms
            atom_positions = protein_dict['shifted_positions'][bool_backbone]

        else: # it was 'sidechain' filter, so grab sidechain atoms
            backbone_atom_set = np.array(['O', 'C', 'CA', 'N'])
            sidechain_atom_set = np.array([atom for atom in protein_dict['atom_type_set'] \
                                           if atom not in backbone_atom_set])

            for index, sidechain_atom in enumerate(sidechain_atom_set):
                if index == 0:
                    # create the first sidechains boolean array, will be edited
                    bool_sidechains = protein_dict['atom_types'] == sidechain_atom
                else:
                    # single boolean array for the current sidechain atom
                    bool_atom = protein_dict['atom_types'] == sidechain_atom

                    # sum this boolean array with the master boolean array
                    bool_sidechains += bool_atom

            # grab all sidechain atom positions
            atom_positions = protein_dict['shifted_positions'][bool_sidechains]

    else: # it was a residue property channel
        acidic_residues = np.array(['ASP', 'GLU'])
        basic_residues = np.array(['LYS', 'ARG', 'HIS'])
        polar_residues = np.array(['GLN', 'ASN', 'HIS', 'SER', 'THR', 'TYR', 'CYS'])
        nonpolar_residues = np.array(['GLY', 'ALA', 'VAL', 'LEU', \
                                        'ILE', 'MET', 'PRO', 'PHE', 'TRP'])
        amphipathic_residues = np.array(['TRP', 'TYR', 'MET'])
        charged_residues = np.array(['ARG', 'LYS', 'ASP', 'GLU'])
        # custom_residues = something
        property_dict = {'acidic':acidic_residues, 'basic':basic_residues,\
                         'polar':polar_residues, 'nonpolar':nonpolar_residues,\
                         'amphipathic':amphipathic_residues, 'charged':charged_residues}

        atom_positions = atoms_from_residues(protein_dict, property_dict[channel])

    return atom_positions

def atoms_from_residues(protein_dict, residue_list):
    """
    This function finds all the atoms in a protein that are members of any residues
    in a list of residues.
    Parameters:
        protein_dict (dict, required): The dictionary of the protein, returned from
            load_pdb()
        residue_list (list-like, required): The list of residues whose atoms we are
            finding coordinates for
    """
    # construct the appropriate boolean array to index the atoms in the protein_dict
    for index, residue in enumerate(residue_list):
        if index == 0:
            bool_residue = protein_dict['residues'] == residue
        else:
            bool_residue += protein_dict['residues'] == residue

    atom_positions = protein_dict['shifted_positions'][bool_residue]

    return atom_positions

def voxelize(path, channels=['CA'], path_type='file', bin_size=2.0, num_bins=50, save=False, save_fn='voxels.npy', save_path='./'):
    """
    This function creates a dictionary of tensor fields directly from a pdb file.
    These tensor fields can be plotted, or sent directly into the cnn for
        plotting internals, or sent all the way through a cnn/vae to be used for
        training.
    Parameters:
        path (str, required): path to a .pdb file
        channels (list of strings, optional): The list of atomic channels to be included in
            the output dictionary, one field for every channel.
            Any channels from points 1-4 below may be combined in any order.
            i.e., one could call voxelize with the channels parameter as
            channels=['charged', 'CB', 'GLY', 'polar', ...etc]. Note that voxelization
            for channels containing more atoms will take longer.
            1. any of the following atom types
                ['C' 'CA' 'CB' 'CD' 'CD1' 'CD2' 'CE' 'CE1' 'CE2' 'CE3' 'CG' 'CG1' 'CG2'
                'CH2' 'CZ' 'CZ2' 'CZ3' 'N' 'ND1' 'ND2' 'NE' 'NE1' 'NE2' 'NH1' 'NH2' 'NZ'
                'O' 'OD1' 'OD2' 'OE1' 'OE2' 'OG' 'OG1' 'OH' 'OXT' 'SD' 'SG']
            2. Any canonical residue in the protein, using the three letter residue code, all caps
               (NOTE: the residue must actually exist in the protein)
                e.g., ['LYS', 'LEU', 'ALA']
            3. The 'other' channel options: 'backbone', 'sidechains'
            4. There are 6 channels corresponding to specific types of residues:
                'charged', 'polar', 'nonpolar', 'amphipathic', 'acidic', 'basic'
    Returns:
        dictionary: a dictionary containing a voxelized atomic fields, one for each
        channel requested. Each field has shape = ([1, 1, 50, 50, 50])
    """
    if path_type == 'file':
        protein_dict = load_pdb(path)
        sys.stdout.write('done')
        return make_fields(protein_dict, channels=channels, bin_size=bin_size, num_bins=num_bins)

    # ------------FOLDER FUNCTIONALITY INCOMPLETE---------------
    elif path_type == 'folder':
        fields = []
        pdb_fns = os.listdir(path)
        for j, fn in enumerate(pdb_fns):
            progress = '{}/{} pdbs voxelized ({}%)'.format(j, len(pdb_fns), \
                        round(j / len(pdb_fns) * 100, 2))
            sys.stdout.write('\r'+progress)
            protein_dict = load_pdb(os.path.join(path, fn))
            field, bins = make_fields(protein_dict, channels=channels, bin_size=bin_size, num_bins=num_bins)
            channel_list = []
            for channel in channels:
                channel_list.append(field[channel].reshape(1,
                                                           bins,
                                                           bins,
                                                           bins))
            field = np.concatenate(channel_list).reshape(1,len(channels),
                                                         bins, bins, bins)
            fields.append(field)
        sys.stdout.write("\r\033[K")
        out_statement = 'voxelization complete!\n'
        sys.stdout.write('\r'+out_statement)
        fields = np.concatenate(fields)

        if save:
            np.save(os.path.join(save_path, save_fn), fields)
        else:
            return fields

In [86]:
file_name = '1a1e_ligand.mol2'
myprotein_dict = load_pdb(file_name)
shift_coordinates = shift_coords(myprotein_dict)

In [89]:
voxel = voxelize(file_name, channels=['heavyvalence'], bin_size=2.0,num_bins=50)

donetorch.Size([73])
4
torch.Size([125000, 73])
torch.Size([50, 50, 50])


In [16]:
field_dict = make_fields(myprotein_dict, channels = ['CA'], bin_size=2.0,num_bins=50)

from cnns4qspr import visualizer
plottable=plot_field_test(field_dict['CA'],num_bins=50, show=False)
plottable.show()

atom
torch.Size([2])
torch.Size([125000, 2])
torch.Size([50, 50, 50])


NameError: name 'plot_field_test' is not defined

In [17]:
field_dict_20 = make_fields(myprotein_dict, channels = ['CA'], bin_size=2.0,num_bins=50)
plottable=plot_field_test(field_dict_20['CA'],num_bins=50, show=False)
plottable.show()

atom
torch.Size([2])
torch.Size([125000, 2])
torch.Size([50, 50, 50])


NameError: name 'plot_field_test' is not defined

In [ ]:
smarts_list = [
                '[#6+0!$(*~[#7,#8,F]),SH0+0v2,s+0,S^3,Cl+0,Br+0,I+0]',
                '[a]',
                '[!$([#1,#6,F,Cl,Br,I,o,s,nX3,#7v5,#15v5,#16v4,#16v6,*+1,*+2,*+3])]',
                '[!$([#6,H0,-,-2,-3]),$([!H0;#7,#8,#9])]',
                '[r]'
            ]

smart_filters = np.array(['hydrophobic', 'aromatic', 'acceptor', 'donor',
                             'ring'])

channel = 'ring'
# smarts_list[smart_filters==channel)]
smarts_list_entry = smarts_list[np.int(np.where(smart_filters == channel)[0])]
molecule = next(pybel.readfile('mol2', '1a1e_ligand.mol2'))
pattern = pybel.Smarts(smarts_list_entry)
atoms_with_prop = np.array(list(*zip(*pattern.findall(molecule))),
                                       dtype=int) - 1
atoms_with_prop

In [ ]:
path = '../../PDBbind/test-set/1a1e/1a1e_ligand.mol2'
file_type = path.split('.')[-1]
file_type

In [ ]:
named_prop = np.array(['hyb', 'heavyvalence', 'heterovalence', 'partialcharge'])

channel = 'partialcharge'

print(index)
prop = []
for atom in pocket:
    #atom.__getattribute__(prop)
    prop.append(atom.__getattribute__(named_prop[np.int(np.where(named_prop == channel)[0])]))

prop = np.array(prop)

normalized_prop =  (prop-min(prop))/(max(prop)-min(prop))
norm_prop = normalized_prop.reshape(73,1)
normalized_prop

In [ ]:

"""
This module contains functions to plot atomic density fields before
they go into a model, as well as what the density fields have been
transformed into at certain points within the model.
"""
import torch
import numpy as np
import pandas as pd
import plotly.express as px
#from cnns4qspr.featurizer import load_cnn

def outer_block1_hook(module, input_, output):
    global outer_block1_out
    outer_block1_out = output


def outer_block2_hook(module, input_, output):
    global outer_block2_out
    outer_block2_out = output


def outer_block3_hook(module, input_, output):
    global outer_block3_out
    outer_block3_out = output


def outer_block4_hook(module, input_, output):
    global outer_block4_out
    outer_block4_out = output


def outer_block5_hook(module, input_, output):
    global outer_block5_out
    outer_block5_out = output

def plot_field_test(
        field,
        num_bins=50,
        color='deep',
        threshold=0.2,
        alpha=0.7,
        show=True,
        title=''):
    """
    This function takes a tensorfield and plots the field density in 3D
    space. The field describes an atomic "density" at each voxel.

    Parameters:
        field (pytorch tensor, required): A field from a field
            dictionary that was output by
            either `voxelize` or `make_fields`.

        color (str, optional): The color scheme to plot the field. Any
            of the Plotly continuous color schemes. 'deep' and 'ice_r'
            are recommended as good baselines.

        threshold (float, optional): The threshold intensity that a
            voxel must have in order to be included in the plot.

        alpha (float, optional): Amount of transparency to use in
            plotted marks.

        show (boolean, optional): Whether to show the plot. If false,
            the plotly fig object is returned.

    Returns:
        plotly figure object: If show=False, a plotly figure object is
        returned
    """
    cube = field

    cube /= cube.max()

    cubelist = []
    xval = np.linspace(-len(cube[0]) + 1, len(cube[0]) - 1, num_bins)
    yval = np.linspace(-len(cube[0]) + 1, len(cube[0]) - 1, num_bins)
    zval = np.linspace(-len(cube[0]) + 1, len(cube[0]) - 1, num_bins)

    # make a dataframe of x,y,z,intensity for each point in the cube
    # to do this have to loop through the cube
    for i, xval2 in enumerate(xval):

        for j, yval2 in enumerate(yval):

            for k, zval2 in enumerate(zval):

                cubelist.append([xval2, yval2, zval2, cube[i][j][k]])

    cube_df = pd.DataFrame(cubelist, columns=['x', 'y', 'z', 'intensity'])

    # only show the voxels with some intensity
    cube_df = cube_df[cube_df['intensity'] > threshold]

    fig = px.scatter_3d(cube_df, x='x', y='y', z='z',
                        color='intensity', opacity=alpha,
                        color_continuous_scale=color)
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[-num_bins / 2.0, num_bins / 2.0]),
            yaxis=dict(range=[-num_bins / 2.0, num_bins / 2.0]),
            zaxis=dict(range=[-num_bins / 2.0, num_bins / 2.0])
        ), title=title
    )
    if show:
        fig.show()
        figret = None
    else:
        figret = fig
    return figret
